# 02_1_crawlingPTAUTM

#### Crawling Data from CNBC Indonesia


<p>this code memperlihatkan crawling data dari situs ([cnbn Indonesia](https://www.cnbcindonesia.com)) dimana kita mengambil 10.000 berita dari 10 kategori yaitu market, news, enterpreneur, sharia, tech, lifestile, opini, my money, cuap cuap cuan and reseach.</p>

hasil crawling  dapat diakses di [utputCNBCIndonesia](https://drive.google.com/file/d/1U9T9LPNaWMZH0KXkymcK4Qha8bt1UXWg/view?usp=sharing)

In [ ]:
! pip install requests
! pip install bs4
! pip install pandas

In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from urllib.parse import urljoin

In [18]:
def scrape_cnbc_indonesia():
    """
    Fungsi untuk melakukan scraping data dari CNBC Indonesia
    """
    # Header untuk mensimulasikan browser
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    # Daftar kategori berdasarkan value dari dropdown
    categories = [
        {"id": 1, "name": "MARKET", "value": "market/5"},
        {"id": 2, "name": "NEWS", "value": "news/3"},
        {"id": 3, "name": "ENTREPRENEUR", "value": "entrepreneur/9"},
        {"id": 4, "name": "SHARIA", "value": "syariah/10"},
        {"id": 5, "name": "TECH", "value": "tech/12"},
        {"id": 6, "name": "LIFESTYLE", "value": "lifestyle/11"},
        {"id": 7, "name": "OPINI", "value": "opini/13"},
        {"id": 8, "name": "MY MONEY", "value": "mymoney/71"},
        {"id": 9, "name": "CUAP CUAP CUAN", "value": "cuap-cuap-cuan/78"},
        {"id": 10, "name": "RESEARCH", "value": "research/127"}

    ]

    # Data yang akan dikumpulkan
    data = {
        "id": [],
        "kategori_id": [],
        "kategori": [],
        "judul": [],
        "penulis": [],
        "tanggal_terbit": [],
        "isi_berita": [],
        "url": []
    }

    # Loop melalui setiap kategori
    for category in categories:
        print(f"Mengambil data dari kategori: {category['name']} (ID: {category['id']})")

        # Counter untuk artikel dalam kategori ini
        article_counter = 1

        # Loop melalui halaman (1 sampai 100)
        for page in range(1, 101):
            # Membuat URL untuk setiap halaman
            # if page == 1:
            #     url = f"https://www.cnbcindonesia.com/indeks/{category['value']}"
            # else:
            url = f"https://www.cnbcindonesia.com/{category['value'].split('/')[0]}/indeks/{category['value'].split('/')[1]}?page={page}"

            print(f"Mengambil halaman: {url}")

            try:
                # Request ke halaman
                response = requests.get(url, headers=headers)
                response.raise_for_status()
                soup = BeautifulSoup(response.content, 'html.parser')

                # Mencari semua artikel
                articles = soup.find_all('article')

                if not articles:
                    print(f"Tidak ada artikel ditemukan di halaman {page}")
                    break

                # Loop melalui setiap artikel
                for article in articles:
                    try:
                        # Mengambil link artikel
                        link = article.find('a')['href']
                        if not link.startswith('http'):
                            link = urljoin('https://www.cnbcindonesia.com', link)

                        # Mengambil data dari detail artikel
                        article_data = scrape_article_detail(link, headers, category)

                        if article_data:
                            # Membuat ID unik: nomor_kategori_nomor_urut_berita
                            article_id = f"{category['id']}_{article_counter}"

                            # Menambahkan data ke dictionary
                            data["id"].append(article_id)
                            data["kategori_id"].append(category['id'])
                            data["kategori"].append(article_data["kategori"])
                            data["judul"].append(article_data["judul"])
                            data["penulis"].append(article_data["penulis"])
                            data["tanggal_terbit"].append(article_data["tanggal_terbit"])
                            data["isi_berita"].append(article_data["isi_berita"])
                            data["url"].append(link)

                            print(f"Berhasil mengambil: {article_id} - {article_data['judul'][:50]}...")

                            # Increment counter
                            article_counter += 1

                        # Delay untuk tidak membebani server
                        time.sleep(1)

                    except Exception as e:
                        print(f"Error saat mengambil artikel: {e}")
                        continue

            except Exception as e:
                print(f"Error saat mengambil halaman {page}: {e}")
                continue

    # Mengubah dictionary menjadi DataFrame
    df = pd.DataFrame(data)
    return df

def scrape_article_detail(url, headers, category):
    """
    Fungsi untuk mengambil detail artikel dari URL tertentu
    """
    try:
        # Request ke halaman artikel
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # Mengambil judul
        judul_elem = soup.select_one('h1.text-32')
        judul = judul_elem.text.strip() if judul_elem else "Tidak ditemukan"

        # Mengambil penulis
        penulis_elem = soup.select_one('div.mb-1.text-base.font-semibold')
        if penulis_elem:
            # Menghapus span dari teks penulis
            for span in penulis_elem.find_all('span'):
                span.decompose()
            penulis = penulis_elem.text.strip()
        else:
            penulis = "Tidak ditemukan"

        # Mengambil tanggal terbit
        tanggal_elem = soup.select_one('div.text-cm.text-gray')
        tanggal_terbit = tanggal_elem.text.strip() if tanggal_elem else "Tidak ditemukan"

        # Mengambil isi berita
        isi_berita = ""
        # Mencari semua paragraf dalam konten artikel
        content_elems = soup.select('div.detail-text p, #detailHead ~ p')
        for elem in content_elems:
            # Skip paragraf yang berisi iklan atau elemen tidak diinginkan
            if elem.find_parents('div', class_=re.compile('ads|advertisement|iklan')):
                continue
            isi_berita += elem.text.strip() + "\n\n"

        # Jika tidak ditemukan dengan selektor di atas, coba cara lain
        if not isi_berita.strip():
            # Coba mengambil dari tag article
            article_elem = soup.find('article')
            if article_elem:
                for p in article_elem.find_all('p'):
                    isi_berita += p.text.strip() + "\n\n"

        # Membersihkan isi berita
        isi_berita = isi_berita.strip()

        return {
            "kategori": category['name'],
            "judul": judul,
            "penulis": penulis,
            "tanggal_terbit": tanggal_terbit,
            "isi_berita": isi_berita
        }

    except Exception as e:
        print(f"Error saat mengambil detail artikel {url}: {e}")
        return None

# Menjalankan scraping
if __name__ == "__main__":
    print("Memulai scraping CNBC Indonesia...")
    hasil_scraping = scrape_cnbc_indonesia()

    if not hasil_scraping.empty:
        # Menyimpan hasil ke CSV
        hasil_scraping.to_csv('cnbc_indonesia_articles.csv', index=False)
        print(f"Berhasil mengambil {len(hasil_scraping)} artikel. Data disimpan ke cnbc_indonesia_articles.csv")

        # Menampilkan preview
        print("\nPreview data:")
        print(hasil_scraping.head())

        # Menampilkan statistik per kategori
        print("\nStatistik per kategori:")
        kategori_stats = hasil_scraping['kategori'].value_counts()
        print(kategori_stats)
    else:
        print("Tidak ada data yang berhasil diambil.")

Streaming output truncated to the last 5000 lines.
Berhasil mengambil: 5_936 - Misteri Pemesan Chip Samsung Senilai Rp 270 Triliu...
Berhasil mengambil: 5_937 - Bos MDI Ventures Ditangkap, Asosiasi Modal Ventura...
Berhasil mengambil: 5_938 - Pendaftaran Internet Murah 100 Mbps Dibuka, Ada di...
Berhasil mengambil: 5_939 - Australia Larang YouTube Ditonton Anak, Diancam De...
Berhasil mengambil: 5_940 - Aturan AI Terbit September 2025, Tak Bisa Sembaran...
Mengambil halaman: https://www.cnbcindonesia.com/tech/indeks/12?page=95
Berhasil mengambil: 5_941 - Bukan BI Checking atau SLIK OJK, Ini Cara Terbaru ...
Berhasil mengambil: 5_942 - Universitas RI Masuk Indeks Risiko Integritas Rise...
Berhasil mengambil: 5_943 - Tips Main Roblox Biar Tetap Aman, Orang Tua Harus ...
Berhasil mengambil: 5_944 - Kasus Korupsi, Eks CEO Tanihub dan Bos MDI Venture...
Berhasil mengambil: 5_945 - Ahli Harvard Ungkap Alasan Pasangan Tidak Punya An...
Berhasil mengambil: 5_946 - China Chaos! Netizen Murka Ga